##Model & Dataset
**Dataset:**
This repository utilizes a specialized computer vision dataset designed for the automated detection of coronary artery stenosis, which refers to the abnormal narrowing or blockage of blood vessels supplying the heart. The dataset consists of X-ray coronary angiography images annotated with bounding boxes to localize these blockages.

The data is structured in the standard YOLO annotation format. It is configured as a single-class detection problem, focusing exclusively on identifying regions labeled as "Stenosis." To facilitate standard machine learning workflows, the dataset is pre-partitioned into distinct training, validation, and testing directories, ensuring rigorous model evaluation and preventing data leakage.

This specific dataset version was processed, formatted, and exported via Roboflow within the axion-bhapu workspace. It is distributed under the Creative Commons Attribution 4.0 International (CC BY 4.0) license. The Roboflow dataset is derived from a publicly available medical archive hosted on Mendeley Data, originally compiled by Danilov et al. (2021) for angiographic stenosis detection.

To integrate this dataset into the training pipeline, the repository includes a data.yaml configuration file.

**Model:** YOLOv26-small (yolo26s.pt) architecture. The selected "small" variant offers a great balance between rapid inference speeds—which is crucial for real-time or near-real-time clinical applications—and high accuracy. Using pre-trained weights allows the model to leverage transfer learning, starting with a strong foundational understanding of visual features before fine-tuning on the complex medical scans.

In [ ]:
%pip install opencv-python-headless matplotlib

print("Installation complete.")

You should consider upgrading via the '/Users/mariaplatek/projects/AXION/YOLOv11/.venv/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/mariaplatek/projects/AXION/YOLOv11/.venv/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
✅ Installation complete.


In [ ]:
import torch
from ultralytics import YOLO
import os

In [ ]:
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"Success! Training on: {gpu_name}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB VRAM")
    device = 0
elif torch.backends.mps.is_available():
    print("Apple Silicon (MPS) detected!")
    device = "mps"
else:
    print("WARNING: GPU not found. Running on CPU!")
    device = "cpu"

🚀 Success! Training on: NVIDIA GeForce RTX 4090
Memory: 25.25 GB VRAM


In [ ]:
data_yaml_path = "dataset/data.yaml"

print(f"Dataset path set to: {data_yaml_path}")

In [ ]:
model = YOLO("yolo26s.pt")

# (Set workers=4 to ensure stability across different OS/Hardware)
print("Starting training process...")
results = model.train(
    data=data_yaml_path,
    epochs=100,
    imgsz=640,
    device=device,
    batch=64,
    workers=4,
    name="stenosis_detection_run",
    project="Model_Training",
    amp=True,
    plots=True,
    exist_ok=True
)
print("Training finished!")

In [ ]:
# Load the best weights from the training run
best_model_path = "Model_Training/stenosis_detection_run/weights/best.pt"

if os.path.exists(best_model_path):
    eval_model = YOLO(best_model_path)

    # Run validation (Forcing the local data path to avoid Vast.ai path errors)
    print("\nRunning Evaluation on Test Set...")
    metrics = eval_model.val(data=data_yaml_path, split='test', plots=True)

    # Calculate F1 Score safely
    p, r = metrics.box.mp, metrics.box.mr
    f1 = 2 * (p * r) / (p + r) if (p + r) > 0 else 0

    print("\n--- FINAL METRICS ---")
    print(f"mAP@50:    {metrics.box.map50:.2%}")
    print(f"mAP@50-95: {metrics.box.map:.2%}")
    print(f"Precision: {p:.2%}")
    print(f"Recall:    {r:.2%}")
    print(f"F1 Score:  {f1:.2%}")
else:
    print("best.pt not found. Run the training cell first.")

In [ ]:
# Export the model for production use
if os.path.exists(best_model_path):
    print("Exporting model to ONNX format...")
    export_path = eval_model.export(format="onnx")
    print(f"Model successfully exported to: {export_path}")